In [1]:
import h5py, os
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle, random
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
### seed_everythin
seed = 1987
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

### Read data from pickle

In [4]:
filename = 'selected_data.pkl'
outfile = open(filename,'rb')
data = pickle.load(outfile)

In [5]:
len(data['data'])

182330

In [6]:
xdata = np.array(data['data'], dtype = 'float32')
ylablel = np.array(data['labels'], dtype = 'float32')

In [7]:
xdata.shape, ylablel.shape

((182330, 50, 3), (182330,))

### Train test split

In [8]:
### Split data
X_train, X_val, y_train, y_val = train_test_split(xdata, ylablel, test_size=0.2)
X_train.shape, X_val.shape

((145864, 50, 3), (36466, 50, 3))

### Normalize data

In [9]:
xscalers = {}
for i in range(xdata.shape[2]):
    xscalers[i] = StandardScaler()
    X_train[:, :, i] = xscalers[i].fit_transform(X_train[:, :, i])

yscaler = StandardScaler()
y_train = yscaler.fit_transform(y_train.reshape(-1, 1))

In [10]:
scalers = {}
for i in range(xdata.shape[2]):
    X_val[:, :, i] = xscalers[i].transform(X_val[:, :, i])

y_val = yscaler.transform(y_val.reshape(-1, 1))

In [11]:
y_val.max(), y_train.max()

(6.706501, 7.02627)

In [12]:
# scalers = {}
# for i in range(xdata.shape[2]):
#     scalers[i] = StandardScaler()
#     xdata[:, :, i] = scalers[i].fit_transform(xdata[:, :, i])
    
# yscaler = StandardScaler()
# ylablel = yscaler.fit_transform(ylablel.reshape(-1, 1))

### Plot seismogram

In [13]:
p = figure(plot_width=600, plot_height=250)
x = np.arange(50)
i = 10
p.line(x, X_train[i][:, 0], line_width=2, color = 'red')
p.line(x, X_train[i][:, 1], line_width=2, color = 'blue')
p.line(x, X_train[i][:, 2], line_width=2, color = 'black')
show(p)

### Baseline Model

In [57]:
inputs = tf.keras.Input(shape=(50,3))
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(128, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(64, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Dense(16, activation=tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [58]:
model.summary()

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 50, 3)]           0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 150)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               19328     
_________________________________________________________________
dense_25 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 16)                1040      
_________________________________________________________________
dense_27 (Dense)             (None, 1)               

In [59]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
              loss="mean_squared_error",
#               metrics=["mean_absolute_error"]
             )

In [60]:
history = model.fit(x=X_train,
    y = y_train,
    batch_size=512,
    epochs=200,
    validation_data=(X_val, y_val))

Epoch 1/200
285/285 [==============================] - 0s 2ms/step - loss: 1.2677 - val_loss: 0.9971
Epoch 2/200
285/285 [==============================] - 0s 2ms/step - loss: 1.3710 - val_loss: 0.9777
Epoch 3/200
285/285 [==============================] - 0s 2ms/step - loss: 1.7746 - val_loss: 0.9695
Epoch 4/200
285/285 [==============================] - 0s 2ms/step - loss: 1.1450 - val_loss: 0.9692
Epoch 5/200
285/285 [==============================] - 0s 2ms/step - loss: 2.3334 - val_loss: 0.9659
Epoch 6/200
285/285 [==============================] - 0s 2ms/step - loss: 2.3664 - val_loss: 0.9516
Epoch 7/200
285/285 [==============================] - 0s 2ms/step - loss: 1.0877 - val_loss: 0.9501
Epoch 8/200
285/285 [==============================] - 0s 2ms/step - loss: 1.0513 - val_loss: 0.9444
Epoch 9/200
285/285 [==============================] - 0s 2ms/step - loss: 1.8922 - val_loss: 0.9374
Epoch 10/200
285/285 [==============================] - 0s 2ms/step - loss: 0.9891 - val_lo

In [61]:
p = figure(plot_width=600, plot_height=250)
x = np.arange(len(history.history['loss']))
p.line(x, history.history['loss'], line_width=2, color = 'blue', legend_label = 'Train MSE')
p.line(x, history.history['val_loss'], line_width=2, color = 'red', legend_label = 'Val MSE')
show(p)